In [30]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook as tqdm

from sklearn.decomposition import NMF
from scipy.sparse.linalg import svds

#set the max columns to none
pd.set_option('display.max_columns', None)

## I will perform matrix factorization to the interaction matrix

In [31]:
# Read interaction matrix pickle file
size = 'small'
interaction_matrix_file_path = f'./files/pickle/interaction_matrix_{size}.pkl'
interaction_matrix = pd.read_pickle(interaction_matrix_file_path)
print(interaction_matrix.shape)

(15143, 8786)


In [32]:
interaction_matrix.head(2)

article_id_fixed,9667501,9676308,9735753,9739065,9756546,9757508,9758025,9759671,9759717,9763634,9764579,9764765,9765067,9423230,9757972,9759345,9763284,9763854,9763923,9764325,9764622,9765175,9765274,9766242,9766264,9766599,9767220,9767242,9767342,9767483,9767507,9767751,9767868,9767948,9768387,9768515,9769380,9769414,9769575,9770541,9701366,9733713,9733845,9736689,9737243,9738490,9738528,9738557,9738569,9740427,9740473,9745698,9745912,9746068,9747404,9747437,9747789,9749025,9749196,9749278,9750772,9752860,9754798,9754882,9755010,9755822,9755849,9756295,9756362,9756399,9757717,9757876,9765871,9765989,9766007,9766022,9767368,9767417,9767551,9767646,9767675,9767741,3218932,4514528,9322996,9685790,9729839,9729988,9730301,9737266,9738303,9738844,9738949,9739035,9739179,9739399,9739837,9740324,9740610,9740747,9741032,9741958,9742172,9742255,9742268,9742423,9743755,9743768,9744477,9744480,9744481,9744501,9745709,9745745,9745793,9746105,9746466,9747039,9747267,9747369,9748183,9748242,9748470,9748509,9748519,9748576,9748767,9748807,9749743,9749760,9750078,9750687,9750749,9751107,9751385,9751531,9751532,9751705,9751706,9751764,9751866,9752375,9752638,9752656,9752846,9753168,9753442,9754042,9754265,9755176,9755224,9755298,9755364,9755911,9755966,9756028,9756047,9756110,9756168,9756695,9757338,9757599,9757952,9758252,9758538,9759398,9759927,9761704,9761914,9762387,9762435,9762439,9765244,9765334,9765336,9765410,9765438,9765481,9765699,9765846,9765955,9766042,9766108,9766428,9766627,9766722,9766805,9767071,9767233,9767474,9768328,9768377,9769459,9769572,9769641,9770533,9690920,9714644,9716537,9725132,9725978,9726836,9728166,9730564,9735579,9736809,9736862,9737199,9737393,9737420,9738801,9738855,9738868,9738974,9739005,9739200,9739282,9739333,9739411,9739471,9739499,9739597,9739634,9739864,9739870,9739888,9740156,9740182,9740383,9740551,9740557,9740652,9740907,9740942,9740952,9740985,9741372,9741523,9741772,9741788,9741802,9742188,9742225,9742226,9742360,9742440,9742459,9742586,9742627,9742635,9742681,9742693,9742770,9743574,9743692,9743733,9743748,9743753,9743792,9743795,9743818,9743846,9743863,9743934,9743935,9743957,9744016,9744055,9744152,9744193,9744213,9744216,9744260,9744733,9744753,9744880,9744884,9744922,9745111,9745130,9745140,9745221,9745367,9745402,9745419,9745450,9745484,9745706,9745733,9745883,9745892,9746258,9746342,9746345,9746364,9746395,9746868,9746984,9747329,9747803,9747962,9747985,9748041,9748248,9748574,9748678,9748792,9748801,9748918,9748947,9748977,9749034,9749036,9749182,9749184,9749634,9749637,9749668,9749756,9749759,9749905,9749938,9750002,9750039,9750389,9750705,9750726,9750789,9750829,9750904,9750948,9750962,9750971,9751143,9751220,9751284,9751290,9751517,9751524,9751564,9751633,9751646,9751688,9751786,9751895,9751897,9751932,9751975,9751982,9752038,9752063,9752071,9752299,9752312,9752366,9752431,9752463,9752731,9752741,9752824,9752984,9752998,9753473,9753503,9753526,9753545,9753584,9754025,9754081,9754087,9754220,9754288,9754365,9754484,9754571,9754598,9754925,9754962,9755025,9755269,9755361,9755430,9755509,9755511,9755897,9755964,9755980,9756081,9756190,9756745,9756899,9757045,9757183,9757191,9757218,9757426,9757676,9757686,9757714,9757716,9757878,9757891,9757936,9758057,9758328,9758374,9758734,9758894,9759025,9759045,9759063,9759219,9759612,9759885,9760067,9760221,9760288,9760334,9760386,9760521,9760672,9760741,9760747,9760944,9761087,9761127,9761288,9761419,9761561,9761626,9761784,9761787,9761924,9761963,9762125,9762145,9762377,9762441,9762520,9762530,9762809,9762952,9762953,9762986,9763120,9763233,9763398,9763401,9763489,9763579,9763702,9763857,9763869,9763977,9764070,9764199,9764361,9764641,9764759,9764822,9765551,9765582,9765641,9765883,9765925,9766015,9766146,9766161,9766178,9766225,9766261,9766279,9766307,9766382,9766419,9766452,9766620,9766635,9766752,9766808,9766912,9766919,9766944,9766949,9767027,9767274,9767336,9767399,9767557,9767604,9767722,9768002,9768062,9768178,9768260,9768722,9768793,9768882

In [22]:
# interaction_matrix = interaction_matrix.replace({np.nan: fillnan_value})
fillnan_value = 'mean_column'
interaction_matrix.fillna(interaction_matrix.mean(), inplace=True)

In [23]:
interaction_matrix.head(2)

article_id_fixed,9667501,9676308,9735753,9739065,9756546,9757508,9758025,9759671,9759717,9763634,9764579,9764765,9765067,9423230,9757972,9759345,9763284,9763854,9763923,9764325,9764622,9765175,9765274,9766242,9766264,9766599,9767220,9767242,9767342,9767483,9767507,9767751,9767868,9767948,9768387,9768515,9769380,9769414,9769575,9770541,9701366,9733713,9733845,9736689,9737243,9738490,9738528,9738557,9738569,9740427,9740473,9745698,9745912,9746068,9747404,9747437,9747789,9749025,9749196,9749278,9750772,9752860,9754798,9754882,9755010,9755822,9755849,9756295,9756362,9756399,9757717,9757876,9765871,9765989,9766007,9766022,9767368,9767417,9767551,9767646,9767675,9767741,3218932,4514528,9322996,9685790,9729839,9729988,9730301,9737266,9738303,9738844,9738949,9739035,9739179,9739399,9739837,9740324,9740610,9740747,9741032,9741958,9742172,9742255,9742268,9742423,9743755,9743768,9744477,9744480,9744481,9744501,9745709,9745745,9745793,9746105,9746466,9747039,9747267,9747369,9748183,9748242,9748470,9748509,9748519,9748576,9748767,9748807,9749743,9749760,9750078,9750687,9750749,9751107,9751385,9751531,9751532,9751705,9751706,9751764,9751866,9752375,9752638,9752656,9752846,9753168,9753442,9754042,9754265,9755176,9755224,9755298,9755364,9755911,9755966,9756028,9756047,9756110,9756168,9756695,9757338,9757599,9757952,9758252,9758538,9759398,9759927,9761704,9761914,9762387,9762435,9762439,9765244,9765334,9765336,9765410,9765438,9765481,9765699,9765846,9765955,9766042,9766108,9766428,9766627,9766722,9766805,9767071,9767233,9767474,9768328,9768377,9769459,9769572,9769641,9770533,9690920,9714644,9716537,9725132,9725978,9726836,9728166,9730564,9735579,9736809,9736862,9737199,9737393,9737420,9738801,9738855,9738868,9738974,9739005,9739200,9739282,9739333,9739411,9739471,9739499,9739597,9739634,9739864,9739870,9739888,9740156,9740182,9740383,9740551,9740557,9740652,9740907,9740942,9740952,9740985,9741372,9741523,9741772,9741788,9741802,9742188,9742225,9742226,9742360,9742440,9742459,9742586,9742627,9742635,9742681,9742693,9742770,9743574,9743692,9743733,9743748,9743753,9743792,9743795,9743818,9743846,9743863,9743934,9743935,9743957,9744016,9744055,9744152,9744193,9744213,9744216,9744260,9744733,9744753,9744880,9744884,9744922,9745111,9745130,9745140,9745221,9745367,9745402,9745419,9745450,9745484,9745706,9745733,9745883,9745892,9746258,9746342,9746345,9746364,9746395,9746868,9746984,9747329,9747803,9747962,9747985,9748041,9748248,9748574,9748678,9748792,9748801,9748918,9748947,9748977,9749034,9749036,9749182,9749184,9749634,9749637,9749668,9749756,9749759,9749905,9749938,9750002,9750039,9750389,9750705,9750726,9750789,9750829,9750904,9750948,9750962,9750971,9751143,9751220,9751284,9751290,9751517,9751524,9751564,9751633,9751646,9751688,9751786,9751895,9751897,9751932,9751975,9751982,9752038,9752063,9752071,9752299,9752312,9752366,9752431,9752463,9752731,9752741,9752824,9752984,9752998,9753473,9753503,9753526,9753545,9753584,9754025,9754081,9754087,9754220,9754288,9754365,9754484,9754571,9754598,9754925,9754962,9755025,9755269,9755361,9755430,9755509,9755511,9755897,9755964,9755980,9756081,9756190,9756745,9756899,9757045,9757183,9757191,9757218,9757426,9757676,9757686,9757714,9757716,9757878,9757891,9757936,9758057,9758328,9758374,9758734,9758894,9759025,9759045,9759063,9759219,9759612,9759885,9760067,9760221,9760288,9760334,9760386,9760521,9760672,9760741,9760747,9760944,9761087,9761127,9761288,9761419,9761561,9761626,9761784,9761787,9761924,9761963,9762125,9762145,9762377,9762441,9762520,9762530,9762809,9762952,9762953,9762986,9763120,9763233,9763398,9763401,9763489,9763579,9763702,9763857,9763869,9763977,9764070,9764199,9764361,9764641,9764759,9764822,9765551,9765582,9765641,9765883,9765925,9766015,9766146,9766161,9766178,9766225,9766261,9766279,9766307,9766382,9766419,9766452,9766620,9766635,9766752,9766808,9766912,9766919,9766944,9766949,9767027,9767274,9767336,9767399,9767557,9767604,9767722,9768002,9768062,9768178,9768260,9768722,9768793,9768882

In [24]:
# ---------(NOT USED)-----------------
# # Apply matrix factorization 
# nmf = NMF(n_components=5)
# user_matrix = nmf.fit_transform(interaction_matrix)
# article_matrix = nmf.components_.T


# Perform Singular Value Decomposition (SVD) on the user-article interaction matrix
k = 300  # Number of latent factors (embedding dimensions)

U, sigma, Vt = svds(interaction_matrix.values, k=k)  # Perform SVD, obtaining U, sigma, Vt

# Diagonalize the singular values sigma to create a diagonal matrix
sigma_diag = np.diag(sigma)

# Reconstruct the user and article embeddings
user_matrix = np.dot(U, np.sqrt(sigma_diag))  # User embeddings
article_matrix = np.dot(np.sqrt(sigma_diag), Vt).T  # Article embeddings

In [10]:
print(user_matrix.shape)
print(article_matrix.shape)

(15143, 300)
(8786, 300)


In [26]:
column_lst = interaction_matrix.columns.tolist()

user_matrix_df = pd.DataFrame(user_matrix, index=interaction_matrix.index)
article_matrix_df = pd.DataFrame(article_matrix, index=column_lst)

In [27]:
user_matrix_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10068,0.094488,-0.008571,-0.030623,-0.181643,0.081566,0.156399,0.000708,0.030428,-0.075494,0.009949,-0.141942,-0.168969,0.049588,0.043216,0.086877,-0.000999,0.063363,-0.002143,0.085487,0.029280,0.067820,-0.097055,0.066549,-0.028127,0.020751,0.018563,0.044981,-0.016134,-0.107023,-0.133634,0.045677,0.105237,-0.002192,0.018731,-0.041087,-0.061399,-0.043645,0.007055,-0.050987,0.006741,0.121876,0.028359,-0.052488,0.111173,0.101067,-0.083578,-0.082304,0.09136,0.106685,-0.076693,0.090356,-0.070314,-0.033077,-0.047215,0.066373,0.102757,-0.062919,-0.006277,-0.030818,0.016764,0.037822,-0.108239,-0.234955,0.030588,-0.000755,0.115009,-0.044087,0.034473,-0.043304,0.010837,-0.060793,0.024912,-0.063153,0.015910,0.034887,0.066819,-0.117846,0.004655,-0.010049,0.009412,0.048355,-0.088411,-0.041045,0.026669,0.009128,-0.051543,0.020651,-0.085802,-0.087090,0.067141,-0.036423,-0.072516,-0.050078,-0.127700,-0.020947,-0.051116,0.067181,-0.059133,0.013904,0.003144,-0.130916,0.035667,-0.117354,-0.143786,0.130476,0.000469,-0.031028,-0.042015,0.016732,-0.084309,0.006496,-0.139238,0.013774,-0.143693,-0.057124,-0.059394,0.007939,0.057337,0.031646,0.139379,0.021311,0.041013,-0.137396,-0.099196,0.036802,0.093321,-0.082035,0.132466,-0.156547,-0.061388,-0.020293,-0.192631,-0.020041,-0.048272,-0.075783,0.094582,0.048048,-0.077269,0.019942,0.098308,0.011016,-0.004019,-0.000563,-0.041305,0.004572,0.004126,0.089651,-0.053939,-0.027933,-0.044448,-0.201787,0.131159,-0.111758,-0.066503,0.009029,-0.102119,0.115862,0.024572,-0.071196,-0.097590,0.041870,0.042690,-0.113435,0.016165,0.133260,0.022592,-0.033054,0.045243,-0.087038,0.043104,-0.032599,-0.017315,-0.061016,-0.039153,0.024147,-0.076586,-0.174045,0.192233,-0.133541,-0.080993,0.109154,0.143926,-0.099513,-0.036072,-0.014970,-0.019060,-0.074498,-0.038063,-0.137244,0.074051,0.071677,0.186974,-0.054223,0.079932,0.009849,0.047581,-0.084524,-0.021383,-0.047456,0.013187,0.083169,-0.021910,0.067999,-0.007655,-0.099883,0.021170,0.051097,0.020482,-0.010614,-0.076503,-0.020301,0.025767,-0.014721,0.001137,-0.021015,0.016204,0.040502,0.013303,-0.015632,0.066394,-0.056052,0.005137,-0.024415,0.200385,-0.063402,0.076459,-0.012060,-0.003165,-0.034505,0.032041,0.028378,0.001293,-0.058495,-0.023185,-0.015730,0.065571,0.072234,-0.118631,-0.100617,-0.041963,0.018448,-0.061630,-0.018741,0.041161,-0.051140,0.034991,0.067725,-0.080013,-0.023620,0.048822,-0.002078,-0.077721,-0.021926,0.099771,0.035318,-0.086356,-0.002746,-0.018445,-0.024409,0.032838,0.006081,0.063029,0.004835,-0.045704,-0.030091,-0.069371,0.057735,-0.068971,-0.069022,0.042484,-0.040247,0.002481,0.058

In [28]:
article_matrix_df.head(2)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
9667501,-1.688894,-0.747952,1.547553,3.24900,1.221855,-0.718673,0.955976,-0.386779,-0.430974,-0.583810,-1.763783,-0.125578,-1.038751,0.817092,2.754225,2.326996,0.034407,0.915141,1.790181,2.004580,-1.542429,-0.077827,-0.833245,-1.213941,-1.307744,-1.359922,0.575467,0.016192,0.832632,1.568775,-2.085927,2.131803,0.252611,-2.215449,0.052079,-1.649706,-2.912574,-0.236563,-1.827604,1.710054,-1.437027,-1.340964,-1.098647,0.960227,-3.970615,-1.020327,2.255185,-1.952874,-1.600306,3.254197,2.016496,1.609768,3.516734,-0.375805,1.058410,3.369450,-0.013000,0.442277,-0.221409,1.202397,-0.457140,0.812701,2.731519,-2.779751,0.015497,2.466745,0.026530,-1.573700,1.015463,-1.919058,1.362078,0.373038,-0.728195,0.504968,-0.812594,3.065573,-1.126313,3.334903,3.268370,0.943117,-2.979169,1.059759,-0.145234,3.585376,2.566424,-0.49977,0.562088,-0.538614,-0.017842,0.736325,0.337622,2.494074,1.981019,0.442497,1.631144,4.717719,4.383162,3.856101,3.561191,3.160898,-2.295416,1.755880,1.88313,3.362426,2.926027,-2.363252,2.397295,-3.389389,-1.509757,4.579805,0.674118,-0.549454,1.118193,2.314266,-1.324277,0.497734,1.405258,-2.866366,-2.977760,1.340836,-2.909285,-0.549290,-0.009316,3.056962,0.804896,-1.667702,-6.781677,-3.018560,0.582739,0.475589,0.402511,-0.435446,-0.215186,4.612305,3.829144,-4.329767,-1.435140,-2.696267,1.625895,0.298050,-0.900320,1.737175,-3.330948,1.176093,-0.289134,1.940525,-3.162014,-1.763049,-3.282994,-3.671757,4.031441,0.619505,5.866881,-4.034454,0.895831,1.488128,0.828319,0.288697,3.618902,-2.957388,3.130707,-4.278840,1.659398,1.215824,-0.511892,-1.143089,-2.533061,-0.378639,0.198106,-0.932552,0.325365,-2.380790,-6.300557,-4.356179,-2.283910,4.648898,2.369454,-3.370497,2.766558,2.089272,-2.984031,-2.472826,1.430016,3.226986,-0.334191,4.840401,2.180416,3.558867,-0.418077,1.106769,3.516610,-6.275592,-4.214680,4.568689,-1.337553,-1.626598,0.088382,3.908596,-9.139940,3.320038,4.864229,-4.148836,-1.120495,-8.689970,1.657335,3.416138,-5.77656,-3.016982,0.583925,-8.031446,6.863723,1.491368,-4.897030,0.627624,2.164030,4.204651,3.048534,2.536285,4.765699,-3.533528,-0.492174,-12.732841,1.419502,3.231224,-3.438502,-4.100478,4.127109,5.196037,4.816332,-11.811063,6.745848,-4.656003,2.800297,0.720422,5.093639,-2.675271,-4.353692,0.464365,0.644997,-9.749830,6.748512,-7.733742,14.822931,-4.150659,-19.148777,-0.678659,2.420508,1.801613,-1.818077,-11.639871,2.751663,29.981276,12.563950,-22.735058,-18.204384,19.112640,14.816937,-9.730502,19.852415,-5.654216,-6.16245,3.344040,25.878321,-0.313127,-15.023375,27.395020,-22.993607,18.465363,22.635874,-8.651117,15.466209,4.811437,6.177443,9.566052,2.485328,-10.107677,8.165385,-7.104902,0.669133,-7.378037,-0.183360,-2.703204,9.726486,0.570019,-3.787957,-4.681210,-3.630626,5.078165,-0.077333,1.324749,-2.689918,-2.468818,0.981329,2.817648,0.811049,-7.744445,7.215307,-5.138723,-0.356481,-7.367726
9676308,0.153221,-0.033801,0.139975,-0.00176,0.1

In [29]:
# Save the cosine similarity matrix to pickle file cause is very big

user_matrix_df_file_path = f'./files/pickle/user_matrix_{size}_fillnan_{str(fillnan_value)}.pkl'
article_matrix_df_file_path = f'./files/pickle/article_matrix_{size}_fillnan_{str(fillnan_value)}.pkl'

user_matrix_df.to_pickle(user_matrix_df_file_path)
article_matrix_df.to_pickle(article_matrix_df_file_path)

## Other

In [5]:
# # Define a function to convert strings back to lists
# def str_to_list(s):
#     try:
#         temp_lst = s.replace('[', '').replace(']','').replace(' ', '').split(',')
        
#         return [float(x) for x in temp_lst]
#     except:
#         return s


# for col in tqdm(interaction_matrix.columns.to_list()):
#     interaction_matrix[col] = interaction_matrix[col].apply(str_to_list)

/var/folders/ld/c07vs56s18bc0h2krlvl0myr0000gn/T/ipykernel_2337/1990072283.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for col in tqdm(interaction_matrix.columns.to_list()):


  0%|          | 0/4248 [00:00<?, ?it/s]